In [1]:
import json
import time
import torch
import transformers
import wandb
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
from transformers import BitsAndBytesConfig


# nltk 다운로드 (최초 1회)
nltk.download('punkt')

# wandb 초기화
wandb.init(project="hangae_99_8", name="8B_4bit_quantization")

# corpus.json 파일 로드 (UTF-8 인코딩)
with open("./corpus.json", "r", encoding="utf-8") as f:
    evaluation_data = json.load(f)

# 10개만 추론
evaluation_data = evaluation_data[:10]

# 허깅페이스 방식으로 모델 및 파이프라인 초기화
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


generator = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=model_id,
    device_map="auto",
    model_kwargs={"torch_dtype": torch.bfloat16},
    quantization_config=quantization_config  # 4비트 양자화를 적용합니다.
)
generator.model.eval()

# 시스템 프롬프트 정의
SYSTEM_PROMPT = (
    "You are a helpful AI assistant. Please answer the user's questions kindly. "
    "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요."
)

# 채팅 템플릿을 적용하여 프롬프트를 생성하는 함수
def make_prompt(system_msg: str, user_msg: str) -> str:

    msg = f"""
다음은 유튜브 타임라인 요약 문서입니다. 이 내용을 분석하여 10개의 핵심 용어(KeyPoint)와 설명을 생성하세요.
설명을 생성할 때 영상 위주가 아닌 최대한 전문적이고 구체적으로 추가 설명해주세요. 저는 영상 내용 뿐 아니라 그 용어에 대한 내용을 알고 싶습니다.
중요하지 않더라도 알면 좋을 추가 핵심 용어도 더해주세요.

[타임라인 요약]:
{user_msg}

[가이드]:
- 필드는 `term`과 `description`을 포함하며, 결과는  10개의 키포인트 리스트로 구성됩니다.
- 반드시 JSON 형식에 맞춰 반환하세요.
- 다른 텍스트는 포함하지 마세요.

[예시]:
{evaluation_data[0]['output']}
"""

    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": msg}
    ]
    prompt = generator.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return prompt

# 종료 토큰 설정
terminators = [
    generator.tokenizer.eos_token_id,
    generator.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

print(f'{round(generator.model.get_memory_footprint() / (1024 ** 3), 1)}GB')
wandb.log({"basic_memory": round(generator.model.get_memory_footprint() / (1024 ** 3), 1)})


# 평가 루프: corpus.json에 있는 각 예제에 대해 추론 수행
for idx, example in enumerate(evaluation_data):
    print(f"\n==== Example {idx+1} ====")

    # 사용자 입력과 프롬프트 생성
    user_input = example["input"]
    prompt = make_prompt(SYSTEM_PROMPT, user_input)

    # 입력 토큰 인코딩
    encoded_input = generator.tokenizer(prompt, return_tensors="pt")
    encoded_input = {k: v.to(generator.model.device) for k, v in encoded_input.items()}
    prompt_length = encoded_input["input_ids"].shape[1]

    # 속도 및 메모리 측정을 위한 시작 시간 기록 및 GPU 메모리 초기화
    start_time = time.time()
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats(generator.model.device)

    # 모델 생성: 토큰별 score 반환 포함
    outputs = generator.model.generate(
        **encoded_input,
        max_new_tokens=1024,
        eos_token_id=terminators[0],
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        return_dict_in_generate=True,
        output_scores=True,
    )

    # GPU 동기화 후 종료 시간 기록
    if torch.cuda.is_available():
        torch.cuda.synchronize(generator.model.device)
    end_time = time.time()
    elapsed_time = end_time - start_time

    # GPU 메모리 사용량 측정 (Peak 메모리, MB 단위)
    if torch.cuda.is_available():
        peak_memory = peak_memory = round(torch.cuda.max_memory_allocated(generator.model.device) / (1024 ** 3), 1)

    else:
        peak_memory = 0

    # 생성된 전체 토큰에서 프롬프트 이후의 candidate 토큰 추출
    generated_ids = outputs.sequences[0]
    candidate_ids = generated_ids[prompt_length:]
    candidate_text = generator.tokenizer.decode(candidate_ids, skip_special_tokens=True).strip()

    print("Candidate output:")
    print(candidate_text)
    print(f"Inference Time: {elapsed_time:.2f} seconds")
    print(f"Peak GPU Memory Usage: {peak_memory:.2f} GB")

    # 참조 텍스트: evaluation_data의 "output"을 JSON 문자열로 변환
    reference_text = json.dumps(example["output"], ensure_ascii=False, indent=2)

    # BLEU 점수 계산 (공백 기준 토큰화)
    candidate_tokens = candidate_text.split()
    reference_tokens = reference_text.split()
    bleu = sentence_bleu(
        [reference_tokens],
        candidate_tokens,
        smoothing_function=SmoothingFunction().method1
    )
    print(f"BLEU score: {bleu:.4f}")

    # 로그 확률 계산: 각 생성 토큰에 대해 log‑softmax 후 선택된 토큰의 로그 확률 합산
    log_score = 0.0
    for i, step_scores in enumerate(outputs.scores):
        log_probs = torch.log_softmax(step_scores, dim=-1)
        token_id = candidate_ids[i].unsqueeze(0)  # shape: (1,)
        token_log_prob = log_probs[0, token_id].item()
        log_score += token_log_prob
    print(f"Log probability score (sum): {log_score:.4f}")

    # wandb에 메트릭 로깅
    wandb.log({
        "example_index": idx,
        "BLEU_score": bleu,
        "Inference_time_sec": elapsed_time,
        "Peak_GPU_Memory_GB": peak_memory,
        "Log_probability_score": log_score
    })

wandb.finish()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
wandb: Currently logged in as: iamkimhongil92 (iamkimhongil92-lumenasoft) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


15.0GB

==== Example 1 ====


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Candidate output:
[{'term': '유니티', 'description': '유니티는 게임 및 시뮬레이션 개발에 널리 사용되는 엔진으로, 2D 및 3D 게임을 제작하는 데 필요한 다양한 툴과 기능을 제공합니다. 다양한 플랫폼에 배포할 수 있는 크로스 플랫폼 기능이 특징입니다.'}, 
 {'term': '뱀서라이크', 'description': '뱀서라이크 게임 장르는 플레이어가 자신의 캐릭터를 조종하여 지속적으로 생성되는 적을 물리치고 생존을 추구하는 게임 유형으로, 주로 롤플레잉 요소를 포함합니다.'}, 
 {'term': '게임 매니저', 'description': '게임 매니저 스크립트는 게임의 주요 로직을 관리하는 역할을 합니다. 게임의 흐름, 이벤트, 상태 등을 제어하여 게임의 동작을 원활하게 합니다.'}, 
 {'term': '레벨', 'description': '게임 내에서 플레이어가 도달해야 하는 목표나 상태를 나타내는 단위로, 게임의 난이도와 콘텐츠를 조절하는 중요한 요소입니다.'}, 
 {'term': '몬스터', 'description': '게임에서 플레이어를 공격하는 적 캐릭터로, 다양한 속성과 능력을 가지고 있으며 게임의 난이도를 조절하는 요소입니다.'}, 
 {'term': '오브젝트 풀링', 'description': '오브젝트 풀링은 게임에서 여러 개의 동일한 오브젝트를 효율적으로 관리하고 생성하는 기법으로, 게임의 성능을 최적화하는 데 유용합니다.'}, 
 {'term': '스폰 데이터', 'description': '몬스터의 속성(타입, 소환 타임, 체력, 스피드)을 저장하는 데이터 구조로, 게임 내에서 몬스터의 초기화와 관리를 위해 사용됩니다.'}, 
 {'term': '직렬화', 'description': '데이터를 컴퓨터 내부에서 효율적으로 저장하고 전송하기 위해 구조화된 형태로 변환하는 과정으로, 게임 내 데이터의 효율적인 관리를 위해 사용됩니다.'}, 
 {'term': '애니메이터', 'descripti

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Candidate output:
[{'term': '유니티', 'description': '유니티는 게임 및 시뮬레이션 개발에 널리 사용되는 엔진으로, 2D 및 3D 게임을 제작하는 데 필요한 다양한 툴과 기능을 제공합니다. 다양한 플랫폼에 배포할 수 있는 크로스 플랫폼 기능이 특징입니다.'}, 
{'term': '프리펩', 'description': '프리펩은 유니티에서 게임 오브젝트의 인스턴스를 쉽게 재사용할 수 있도록 도와주는 유용한 기능으로, 오브젝트의 내용과 외형을 설정하고 필요한 스프라이트를 추가하여 빠르게 게임 환경을 구성할 수 있습니다.'}, 
{'term': '충돌 로직', 'description': '충돌 로직은 게임에서 오브젝트가 서로 충돌할 때 발생하는 이벤트를 처리하는 코드로, 충돌 시의 반응을 정의하고 게임의 동작을 제어하는 데 사용됩니다.'}, 
{'term': '스프라이트', 'description': '스프라이트는 2D 게임에서 사용되는 잘라낸 이미지나 그림을 의미하며, 캐릭터, 아이템, 이펙트 등 다양한 요소의 비주얼을 표현할 수 있습니다.'}, 
{'term': '리짓 체이너지 2D', 'description': '리짓 체이너지 2D 컴포넌트는 물리적 상호작용을 위해 오브젝트에 물리적 특성을 부여하는 기능을 제공하며, 중력이나 충돌 감지 등을 설정할 수 있습니다.'}, 
{'term': '캡슐 콜라이더 2D', 'description': '캡슐 콜라이더 2D는 2D 공간에서 오브젝트의 물리적 경계를 정의하는 컴포넌트로, 캐릭터의 충돌 처리를 보다 쉽게 만들어 줍니다.'}, 
{'term': '그래비티 스케일', 'description': '그래비티 스케일은 리짓 체이너지 컴포넌트에서 중력의 영향을 얼마나 받을지를 결정하는 값으로, 0으로 설정하면 중력이 적용되지 않도록 할 수 있습니다.'}, 
{'term': '유니티 허브', 'description': '유니티 허브는 유니티 엔진의 다양한 버전을 관리하고 프로젝트를 생성할 수

BLEU_score,█▁▁▁▁▁▁▁▁▁
Inference_time_sec,▅▁▃▄▄█▃▄▅▄
Log_probability_score,▇▄▁█▁▇▂▂▇▁
Peak_GPU_Memory_GB,▁▇▅▄▇█▆▅▅▅
basic_memory,▁
example_index,▁▂▃▃▄▅▆▆▇█
BLEU_score,0.00879
Inference_time_sec,19.27719
Log_probability_score,-124.72599
Peak_GPU_Memory_GB,16.4
basic_memory,15
